In [1]:
# Data Quality

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
from pymongo import MongoClient
import csv
from geopy.geocoders import Bing
from config import BING_API_KEY

In [3]:
# Load data set into pandas
df = pd.read_csv ("heart_2022_with_nans.csv")

df.shape[0]
# df.shape[1]

445132

In [4]:
def display_unique_values(df):
    for col in df.columns:
        unique_values = df[col].unique()
        print(f"Column {col} has unique values: {unique_values}")

display_unique_values(df)

Column State has unique values: ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Guam' 'Puerto Rico'
 'Virgin Islands']
Column Sex has unique values: ['Female' 'Male']
Column GeneralHealth has unique values: ['Very good' 'Excellent' 'Fair' 'Poor' 'Good' nan]
Column PhysicalHealthDays has unique values: [ 0.  2.  1.  8.  5. 30.  4. 23. 14. nan 15.  3. 10.  7. 25.  6. 21. 20.
 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 19. 18. 22.]
Column Me

In [5]:
# Check the columns
df.columns

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')

In [6]:
# Add coordinates to the dataset
# Define function to get coordinates for a given location (state)
def get_coordinates(state_name):
    geocoder = Bing(api_key=BING_API_KEY)
    try:
        location = geocoder.geocode(state_name + ', USA', exactly_one=True)
        if location:
            return [location.latitude, location.longitude]
        else:
            print('Failed to geocode state:', state_name)
            return None
    except Exception as e:
        print('Error:', e)
        return None

# List of states
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", 
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", 
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", 
    "Massachusetts", "Michigan", "Minnesota", "Mississippi", 
    "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
    "New Jersey", "New Mexico", "New York", "North Carolina", 
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
    "Rhode Island", "South Carolina", "South Dakota", "Tennessee", 
    "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming"
]

# Fetch coordinates for each state
coordinates = {}
for state in states:
    coordinates[state] = get_coordinates(state)

# Function to update CSV file with coordinates
def update_csv_with_coordinates(csv_file, coordinates):
    updated_data = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            state = row['State']
            if state in coordinates:
                row['Latitude'], row['Longitude'] = coordinates[state]
            updated_data.append(row)
    
    # Write updated data to a new CSV file
    with open('updated_' + csv_file, 'w', newline='') as updated_file:
        writer = csv.DictWriter(updated_file, fieldnames=updated_data[0].keys())
        writer.writeheader()
        writer.writerows(updated_data)

# Update CSV file with latitude and longitude
update_csv_with_coordinates('heart_2022_with_nans.csv', coordinates)

In [7]:
# Check the dataset
df = pd.read_csv ("updated_heart_2022_with_nans.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      445132 non-null  object 
 1   Sex                        445132 non-null  object 
 2   GeneralHealth              443934 non-null  object 
 3   PhysicalHealthDays         434205 non-null  float64
 4   MentalHealthDays           436065 non-null  float64
 5   LastCheckupTime            436824 non-null  object 
 6   PhysicalActivities         444039 non-null  object 
 7   SleepHours                 439679 non-null  float64
 8   RemovedTeeth               433772 non-null  object 
 9   HadHeartAttack             442067 non-null  object 
 10  HadAngina                  440727 non-null  object 
 11  HadStroke                  443575 non-null  object 
 12  HadAsthma                  443359 non-null  object 
 13  HadSkinCancer              44

In [8]:
# Check for any duplication and drop them
columns = ['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
           'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
           'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
           'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
           'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
           'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
           'DifficultyConcentrating', 'DifficultyWalking',
           'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
           'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
           'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
           'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
           'HighRiskLastYear', 'CovidPos','Latitude','Longitude']

# Check for duplicates based on the specified columns
num_duplicates = df.duplicated(subset=columns).sum()

# Get the duplicated rows
Duplicated_data = df[df.duplicated(subset=columns, keep=False)]

# Remove duplicate rows based on the specified columns
df_no_duplicates = df.drop_duplicates(subset=columns, keep="first")

# Output the results
print(f"Number of duplicate rows: {num_duplicates}")
print("Duplicated data:")
print(Duplicated_data)
print("DataFrame without duplicates:")
print(df_no_duplicates)

Number of duplicate rows: 157
Duplicated data:
             State     Sex GeneralHealth  PhysicalHealthDays  \
4712        Alaska    Male     Very good                 0.0   
7310        Alaska    Male     Very good                 0.0   
10696      Arizona  Female     Excellent                 0.0   
11503      Arizona  Female     Excellent                 0.0   
26789   California    Male     Excellent                 0.0   
...            ...     ...           ...                 ...   
428370   Wisconsin    Male          Good                 0.0   
431351   Wisconsin    Male          Good                 0.0   
431653   Wisconsin    Male     Excellent                 0.0   
433340     Wyoming  Female          Good                 0.0   
433508     Wyoming  Female          Good                 0.0   

        MentalHealthDays                                    LastCheckupTime  \
4712                 0.0  Within past year (anytime less than 12 months ...   
7310                 0.0  

In [9]:
# Check for NaN values and drop them
for column in df_no_duplicates.columns:
    if df_no_duplicates[column].isnull().any():
        print(f"The column '{column}' has NaN values:")
        # Count of NaN values in each column
        print(df_no_duplicates[column].isna().sum())
    else:
        print (f"Checked")

Checked
Checked
The column 'GeneralHealth' has NaN values:
1193
The column 'PhysicalHealthDays' has NaN values:
10922
The column 'MentalHealthDays' has NaN values:
9062
The column 'LastCheckupTime' has NaN values:
8301
The column 'PhysicalActivities' has NaN values:
1088
The column 'SleepHours' has NaN values:
5448
The column 'RemovedTeeth' has NaN values:
11355
The column 'HadHeartAttack' has NaN values:
3060
The column 'HadAngina' has NaN values:
4400
The column 'HadStroke' has NaN values:
1552
The column 'HadAsthma' has NaN values:
1768
The column 'HadSkinCancer' has NaN values:
3138
The column 'HadCOPD' has NaN values:
2214
The column 'HadDepressiveDisorder' has NaN values:
2807
The column 'HadKidneyDisease' has NaN values:
1921
The column 'HadArthritis' has NaN values:
2628
The column 'HadDiabetes' has NaN values:
1082
The column 'DeafOrHardOfHearing' has NaN values:
20502
The column 'BlindOrVisionDifficulty' has NaN values:
21419
The column 'DifficultyConcentrating' has NaN value

In [10]:
# Drop rows with any NaN values and keep all the columns
df_cleaned = df_no_duplicates.dropna()
# df_cleaned.isna().sum()
df_cleaned.shape[0]

238407

In [11]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238407 entries, 342 to 435820
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      238407 non-null  object 
 1   Sex                        238407 non-null  object 
 2   GeneralHealth              238407 non-null  object 
 3   PhysicalHealthDays         238407 non-null  float64
 4   MentalHealthDays           238407 non-null  float64
 5   LastCheckupTime            238407 non-null  object 
 6   PhysicalActivities         238407 non-null  object 
 7   SleepHours                 238407 non-null  float64
 8   RemovedTeeth               238407 non-null  object 
 9   HadHeartAttack             238407 non-null  object 
 10  HadAngina                  238407 non-null  object 
 11  HadStroke                  238407 non-null  object 
 12  HadAsthma                  238407 non-null  object 
 13  HadSkinCancer              23840

In [12]:
# df_cleaned: Got cleaned w/o columns removal
print (f"df_cleaned has {df_cleaned.shape[0]} rows")
print (f"df_cleaned has {df_cleaned.shape[1]} columns")

df_cleaned has 238407 rows
df_cleaned has 42 columns


In [14]:
# Save df_cleaned to a CSV file
df_cleaned.to_csv('Heart_42_Col.csv', index=False)

In [15]:
# Dropping columns w/ more than 50000 nan: 
columns = [column for column in df_no_duplicates.columns if df_no_duplicates[column].isna().sum() >= 50000]
df_df = df_no_duplicates.drop(columns=columns)
df_df = df_df.dropna()
df_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302498 entries, 342 to 435824
Data columns (total 36 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      302498 non-null  object 
 1   Sex                        302498 non-null  object 
 2   GeneralHealth              302498 non-null  object 
 3   PhysicalHealthDays         302498 non-null  float64
 4   MentalHealthDays           302498 non-null  float64
 5   LastCheckupTime            302498 non-null  object 
 6   PhysicalActivities         302498 non-null  object 
 7   SleepHours                 302498 non-null  float64
 8   RemovedTeeth               302498 non-null  object 
 9   HadHeartAttack             302498 non-null  object 
 10  HadAngina                  302498 non-null  object 
 11  HadStroke                  302498 non-null  object 
 12  HadAsthma                  302498 non-null  object 
 13  HadSkinCancer              30249

In [16]:
# df_df: Columns got removed before dropna
print (f"df_df has {df_df.shape[0]} rows")
print (f"df_df has {df_df.shape[1]} columns")

df_df has 302498 rows
df_df has 36 columns


In [17]:
# Save df_df to a CSV file
df_df.to_csv('Heart_36_Col.csv', index=False)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Calculate the correlation matrix
# correlation_matrix = df_df.corr()

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title('Correlation Heatmap')
# plt.show()

In [ ]:
# Data Distribution

In [ ]:
# numeric_columns = df_no_duplicates.select_dtypes(include=['int64', 'float64']).columns

# # Compute basic statistical measures
# stats_summary = df_no_duplicates[numeric_columns].describe().transpose()

# # Display the statistical summary
# print(stats_summary)

In [ ]:
# from sklearn.preprocessing import StandardScaler, LabelEncoder

# # Select numerical columns for standardization
# numerical_columns = df_df.select_dtypes(include=['int', 'float']).columns

# # Select categorical and binary columns for preprocessing
# categorical_columns = ['State', 'Sex', 'PhysicalActivities', 'AlcoholDrinkers', 'FluVaxLast12']
# binary_columns = ['HadHeartAttack', 'HadAngina', 'HadStroke', 'HadAsthma', 'HadSkinCancer', 
#                   'HadCOPD', 'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis', 
#                   'DeafOrHardOfHearing', 'BlindOrVisionDifficulty', 'DifficultyConcentrating', 
#                   'DifficultyWalking', 'DifficultyDressingBathing', 'DifficultyErrands']

# # Apply label encoding to categorical columns
# label_encoder = LabelEncoder()
# for col in categorical_columns:
#     df_df[col] = label_encoder.fit_transform(df_df[col])

# # Convert binary columns to 0 and 1
# df_df[binary_columns] = df_df[binary_columns].astype(int)

# # Initialize StandardScaler
# scaler = StandardScaler()

# # Fit scaler to the data and transform numerical columns
# df_df[numerical_columns] = scaler.fit_transform(df_df[numerical_columns])

# # Now, the DataFrame should contain only standardized numerical values


In [ ]:
# # Identify binary columns
# binary_columns_1 = [col for col in df_df.columns if df_df[col].nunique() == 2]
# binary_columns_2 = [col for col in df_cleaned.columns if df_cleaned[col].nunique() == 2]

In [ ]:
# # Convert binary columns to 0 and 1
# for col in binary_columns_1:
#     unique_values = df_df[col].unique()
#     if set(unique_values) != {0, 1}:
#         df[col] = df[col].map({unique_values[0]: 0, unique_values[1]: 1})

# for col in binary_columns_2:
#     unique_values = df_cleaned[col].unique()
#     if set(unique_values) != {0, 1}:
#         df_cleaned[col] = df_cleaned[col].map({unique_values[0]: 0, unique_values[1]: 1})

# # Verify conversion
# all_binary = True
# for col in binary_columns_2:
#     unique_values = set(df_cleaned[col].unique())
#     if unique_values != {0, 1}:
#         all_binary = False
#         print(f"Column {col} has non-binary values: {unique_values}")

# if all_binary:
#     print("All binary columns have been successfully converted to 0 and 1.")
# else:
#     print("Some columns still have non-binary values.")


In [ ]:
# def convert_binary_columns(df, binary_columns):
#     for col in binary_columns:
#         unique_values = df[col].dropna().unique()  # Drop NA values for unique check
#         if set(unique_values) != {0, 1}:
#             if len(unique_values) == 2:
#                 df.loc[:, col] = df[col].map({unique_values[0]: 0, unique_values[1]: 1})
#             else:
#                 print(f"Column {col} has unexpected number of unique values: {unique_values}")
#     return df

# df_df = convert_binary_columns(df_df, binary_columns_1)
# df_cleaned = convert_binary_columns(df_cleaned, binary_columns_2)

# # Verify conversion
# def verify_binary_columns(df, binary_columns):
#     all_binary = True
#     for col in binary_columns:
#         unique_values = set(df[col].dropna().unique())  # Drop NA values for unique check
#         if unique_values != {0, 1}:
#             all_binary = False
#             print(f"Column {col} has non-binary values: {unique_values}")
#     return all_binary

# if verify_binary_columns(df_cleaned, binary_columns_2):
#     print("All binary columns have been successfully converted to 0 and 1.")
# else:
#     print("Some columns still have non-binary values.")


In [ ]:
# def convert_to_int(df, binary_columns):
#     for col in binary_columns:
#         # Use .loc to ensure we are modifying the DataFrame in place
#         df.loc[:, col] = df[col].astype('int8')
#     return df
    
# df_df = convert_to_int(df_df, binary_columns_1)
# df_cleaned = convert_to_int(df_cleaned, binary_columns_2)

# # Apply the conversion function
# df_df = convert_to_int(df_df, binary_columns_1)
# df_cleaned = convert_to_int(df_cleaned, binary_columns_2)

# # Verify conversion (no warning should be issued)
# print(df_df.dtypes)
# # print(df_cleaned.dtypes)

In [ ]:
# # Check DataFrame Columns
# print(df_df.columns)

In [ ]:
# # Columns to exclude
# excluded_columns = ['GeneralHealth', 'LastCheckupTime', 'RemovedTeeth', 'HadDiabetes', 
#                     'SmokerStatus', 'ECigaretteUsage', 'RaceEthnicityCategory', 'AgeCategory']

# # Exclude specified columns from the DataFrame
# df_df_subset = df_df.drop(columns=excluded_columns)

# # Calculate the correlation matrix
# correlation_matrix = df_df_subset.corr()

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title('Correlation Heatmap')
# plt.show()


In [ ]:
# # Pair Plot
# sns.pairplot(df_df_subset)
# plt.title('Pair Plot')
# plt.show()

In [ ]:
# categorical_columns = [
#     'GeneralHealth', 'LastCheckupTime', 'RemovedTeeth', 'HadDiabetes', 
#     'SmokerStatus', 'ECigaretteUsage', 'RaceEthnicityCategory', 'AgeCategory'
# ]

# # Apply one-hot encoding
# df_df = pd.get_dummies(df_df, columns=categorical_columns, drop_first=True)

# # Check the column names after one-hot encoding
# print(df_df.columns)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # Select only the numerical columns for standardization
# numerical_columns = ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters', 'WeightInKilograms', 'BMI']
# #
# # Initialize StandardScaler
# scaler = StandardScaler()

# # Fit scaler to the data and transform the numerical columns
# df_df[numerical_columns] = scaler.fit_transform(df_df[numerical_columns])


In [ ]:
# # Check for non-numeric values in each column
# non_numeric_columns = df_df.columns[df_df.dtypes == 'object']
# print("Columns with non-numeric values:", non_numeric_columns)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # Select only the numerical columns for standardization
# numerical_columns = df_df.columns
# # Initialize StandardScaler
# scaler = StandardScaler()

# # Fit scaler to the data and transform the numerical columns
# df_df[numerical_columns] = scaler.fit_transform(df_df[numerical_columns])

In [ ]:
# # Pair Plot
# sns.pairplot(df_df)
# plt.title('Pair Plot')
# plt.show()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Remove non-numeric columns before creating the heatmap
# df_numeric = df_df.select_dtypes(include=['number'])

# # Calculate the correlation matrix
# correlation_matrix = df_numeric.corr()

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title('Correlation Heatmap')
# plt.show()


In [ ]:
# # Calculate the correlation matrix
# correlation_matrix = df_encoded.corr()

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title('Correlation Heatmap')
# plt.show()
